In [4]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

In [5]:
file_path = r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 3 Files\Questions\costs.csv"

In [6]:
costs_df = pd.read_csv(file_path)

In [7]:
costs_df

,Entry,Fixed,Variable
0,1,2490000.0,0.24
1,2,2480000.0,0.63
2,3,1300000.0,0.82
3,4,2220000.0,0.51
4,5,1280000.0,0.36
5,6,1590000.0,0.49
6,7,1330000.0,0.68
7,8,1980000.0,0.29
8,9,1090000.0,0.52
9,10,1370000.0,0.43


In [8]:
m = Model("HealthLink Supplies")

Restricted license - for non-production use only - expires 2025-11-24


In [9]:
n_sites = len(costs_df)

In [10]:
# Warehouse established or not

x = m.addVars(n_sites, vtype=GRB.BINARY, name="x")

In [11]:
# Units stored at each warehouse

y = m.addVars(n_sites, lb=175000, ub=375000, vtype=GRB.CONTINUOUS, name="y")

In [12]:
total_fixed_cost = quicksum(x[i]*costs_df.loc[i, "Fixed"] for i in range(n_sites))

In [13]:
total_variable_cost = quicksum(y[i]*costs_df.loc[i, "Variable"] for i in range(n_sites))

In [14]:
m.setObjective(total_fixed_cost + total_variable_cost, GRB.MINIMIZE)

In [15]:
# Constraints

In [16]:
# At least four locations must be chosen amongst sites 6-16

m.addConstr(quicksum(x[i] for i in range(5, 16)) >= 4, "constraint_sites_6_16")

<gurobi.Constr *Awaiting Model Update*>

In [17]:
# No more than 6 locations must be chosen amongst the even numbered sites

m.addConstr(quicksum(x[i] for i in range(1, n_sites, 2)) <= 6, "constraint_even_sites")

<gurobi.Constr *Awaiting Model Update*>

In [18]:
# If location 1 or 2 is chosen then the sites 5, 6, and 7 cannot be chosen

m.addConstrs((x[0] + x[1] + x[j] <= 1 for j in [4, 5, 6]))

{4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>}

In [19]:
# If any location from 19-22 is chosen, locations 24, 26, and 27 cannot be chosen

m.addConstr(quicksum(x[i] for i in range(18, 22)) + quicksum(x[j] for j in [23, 25, 26]) <= 1, "constraint_19_22_not_24_26_27")

<gurobi.Constr *Awaiting Model Update*>

In [20]:
# If any location from 1-5 is chosen, at least one odd site from 21-27 must be chosen

m.addConstr(quicksum(x[i] for i in range(5)) - quicksum(x[j] for j in range(20, n_sites, 2)) <= 0, "require_odd_21_27_if_1_to_5")

<gurobi.Constr *Awaiting Model Update*>

In [21]:
# The number of locations chosen in 1-14 must equal the number of locations chosen in 15-27

m.addConstr(quicksum(x[i] for i in range(14)) == quicksum(x[i] for i in range(14, n_sites)), "constraint_equal_division")

<gurobi.Constr *Awaiting Model Update*>

In [22]:
# The sum of all units at sites 1-9 and 19-27 must be equal

m.addConstr(quicksum(y[i] for i in range(9)) == quicksum(y[i] for i in range(18, n_sites)), "constraint_equal_units")

<gurobi.Constr *Awaiting Model Update*>

In [23]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 9 rows, 54 columns and 94 nonzeros
Model fingerprint: 0xf4fe4456
Variable types: 27 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 3e+06]
  Bounds range     [1e+00, 4e+05]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 1.393100e+07
Presolve removed 4 rows and 33 columns
Presolve time: 0.00s
Presolved: 5 rows, 21 columns, 50 nonzeros
Variable types: 0 continuous, 21 integer (21 binary)
Found heuristic solution: objective 8811000.0000

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

In [24]:
print("Optimal cost:", m.objVal)

Optimal cost: 8811000.0


In [25]:
for i in range(n_sites):
    if x[i].x > 0.5:  
        print(f"Warehouse established at site {i+1} with {y[i].x} units.")

Warehouse established at site 9 with 175000.0 units.
Warehouse established at site 13 with 175000.0 units.
Warehouse established at site 15 with 175000.0 units.
Warehouse established at site 16 with 175000.0 units.


In [26]:
# 4 sites established in the optimal solution